In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("C:/Users/limst/Documents/github/Recipe_Recommender_System/Data/Raw Data/train_healthy.csv")
df

,recipe_id,groups,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,5315,2,beverages,icy holiday punch,6,1440,54.5,1.0,50.0,1.0,...,2001-04-24,I brought this to an office holiday party and ...,4,3.998201,0.000000,3.912023,0.000000,0.000000,0.000000,1.386294
1,12791,2,beverages,yellow birds,5,3,137.6,1.0,43.0,1.0,...,2001-11-23,This is awesome!! I created a punch with this ...,5,4.924351,0.000000,3.761200,0.000000,0.000000,0.000000,1.386294
2,3208,3,beverages,iced chai,8,0,546.6,13.0,414.0,7.0,...,2001-11-25,"I actually preferred this hot, it's perfect fo...",4,6.303717,2.564949,6.025866,1.945910,2.833213,3.295837,3.637586
3,14562,3,beverages,banana frappe,7,20,637.2,40.0,243.0,7.0,...,2001-11-25,This Frappe is very good with alot of banana f...,5,6.457084,3.688879,5.493061,1.945910,3.367296,4.094345,3.433987
4,13267,3,beverages,very rich hot buttered rum,8,11,265.3,21.0,132.0,5.0,...,2001-12-26,"I had never had Hot Buttered Rum before, but t...",5,5.580861,3.044522,4.882802,1.609438,0.000000,3.784190,2.397895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72252,390252,1,breads,multigrain sandwich bread gluten free dairy...,15,100,218.9,9.0,12.0,9.0,...,2010-09-03,I have never made a gluten free bread before a...,0,5.388615,2.197225,2.484907,2.197225,2.197225,1.098612,2.484907
72253,180718,1,breads,banana raisin muffins,9,30,278.9,15.0,74.0,13.0,...,2010-09-04,Oh My..FABULOUS. I just wish I would have dou...,5,5.630853,2.708050,4.304065,2.564949,2.302585,1.945910,2.708050
72254,213696,1,breads,flat bread gluten free wheat free dairy free ...,7,18,156.2,9.0,2.0,7.0,...,2010-09-05,Somewhere between three and four stars ;-) Thi...,4,5.051137,2.197225,0.693147,1.945910,1.386294,1.098612,1.945910
72255,344211,1,breads,old south pecan cornbread,9,40,185.3,16.0,36.0,6.0,...,2010-09-05,"Sharon, this is such a great cornbread recipe!...",5,5.221976,2.772589,3.583519,1.791759,1.791759,2.944439,1.791759


# One-hot encoding of the category (Normalised data)

In [3]:
X = df[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X = pd.concat([X, pd.get_dummies(X.category, prefix='Cat_')], axis=1).drop('category', axis = 1)
X

,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,Cat__beverages,Cat__breads,Cat__burgers,Cat__desserts,Cat__meat,Cat__pasta,Cat__salads,Cat__seafood,Cat__soup,Cat__vegetables
0,3.998201,0.000000,3.912023,0.000000,0.000000,0.000000,1.386294,1,0,0,0,0,0,0,0,0,0
1,4.924351,0.000000,3.761200,0.000000,0.000000,0.000000,1.386294,1,0,0,0,0,0,0,0,0,0
2,6.303717,2.564949,6.025866,1.945910,2.833213,3.295837,3.637586,1,0,0,0,0,0,0,0,0,0
3,6.457084,3.688879,5.493061,1.945910,3.367296,4.094345,3.433987,1,0,0,0,0,0,0,0,0,0
4,5.580861,3.044522,4.882802,1.609438,0.000000,3.784190,2.397895,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72252,5.388615,2.197225,2.484907,2.197225,2.197225,1.098612,2.484907,0,1,0,0,0,0,0,0,0,0
72253,5.630853,2.708050,4.304065,2.564949,2.302585,1.945910,2.708050,0,1,0,0,0,0,0,0,0,0
72254,5.051137,2.197225,0.693147,1.945910,1.386294,1.098612,1.945910,0,1,0,0,0,0,0,0,0,0
72255,5.221976,2.772589,3.583519,1.791759,1.791759,2.944439,1.791759,0,1,0,0,0,0,0,0,0,0


In [9]:
y = df['groups']
y

0        2
1        2
2        3
3        3
4        3
        ..
72252    1
72253    1
72254    1
72255    1
72256    1
Name: groups, Length: 72257, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 0)

## Multinomial Logistic Regression

In [39]:
# Import 'LogisticRegression' and create a LogisticRegression object
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
# Import RFE and select 15 variables          
rfe = logreg.fit(X_train, y_train)

C:\Users\limst\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
y_pred = rfe.predict(X_test)
print('Accuracy', metrics.accuracy_score(y_test,y_pred)*100,'%')

Accuracy 77.58095765292 %


# One vs One classification

## Linear 

In [13]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)

## Polynomial Kernal

In [14]:
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly_pred = poly.predict(X_test)

## RBF Kernel

In [15]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

## Sigmoid kernel

In [16]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)
sig_pred = sig.predict(X_test)

## Compare results

In [18]:
accuracy_lin_ovo_lg = linear.score(X_test, y_test)
accuracy_poly_ovo_lg = poly.score(X_test, y_test)
accuracy_rbf_ovo_lg = rbf.score(X_test, y_test)
accuracy_sig_ovo_lg = sig.score(X_test, y_test)
print('Accuracy Linear Kernel:', accuracy_lin_ovo_lg)
print('Accuracy Polynomial Kernel:', accuracy_poly_ovo_lg)
print('Accuracy Radial Basis Kernel:', accuracy_rbf_ovo_lg)
print('Accuracy Sigmoid Kernel:', accuracy_sig_ovo_lg)

Accuracy Linear Kernel: 0.8006504290063658
Accuracy Polynomial Kernel: 0.9218101300858013
Accuracy Radial Basis Kernel: 0.9707998892886798
Accuracy Sigmoid Kernel: 0.3855521727096596


In [19]:
cm_lin = confusion_matrix(y_test, linear_pred)
cm_poly = confusion_matrix(y_test, poly_pred)
cm_rbf = confusion_matrix(y_test, rbf_pred)
cm_sig = confusion_matrix(y_test, sig_pred)
print('Linear:\n', cm_lin)
print('Polynomial:\n', cm_poly)
print('RBF Kernel:\n', cm_rbf)
print('Sigmoid:\n', cm_sig)

Linear:
 [[6551  864   55]
 [ 972 3376  361]
 [  35  594 1644]]
Polynomial:
 [[7173  291    6]
 [ 375 4191  143]
 [   5  310 1958]]
RBF Kernel:
 [[7377   93    0]
 [ 156 4511   42]
 [  50   81 2142]]
Sigmoid:
 [[5416    0 2054]
 [3925    1  783]
 [2118    0  155]]


# One vs Rest classification

## Linear 

In [20]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr').fit(X_train, y_train)
linear_pred = linear.predict(X_test)

## Polynomial Kernal

In [21]:
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovr').fit(X_train, y_train)
poly_pred = poly.predict(X_test)

## RBF Kernel

In [22]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovr').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

## Sigmoid kernel

In [23]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovr').fit(X_train, y_train)
sig_pred = sig.predict(X_test)

## Compare results

In [24]:
accuracy_lin_ovr_lg = linear.score(X_test, y_test)
accuracy_poly_ovr_lg = poly.score(X_test, y_test)
accuracy_rbf_ovr_lg = rbf.score(X_test, y_test)
accuracy_sig_ovr_lg = sig.score(X_test, y_test)
print('Accuracy Linear Kernel:', accuracy_lin_ovr_lg)
print('Accuracy Polynomial Kernel:', accuracy_poly_ovr_lg)
print('Accuracy Radial Basis Kernel:', accuracy_rbf_ovr_lg)
print('Accuracy Sigmoid Kernel:', accuracy_sig_ovr_lg)

Accuracy Linear Kernel: 0.8006504290063658
Accuracy Polynomial Kernel: 0.9218101300858013
Accuracy Radial Basis Kernel: 0.9707998892886798
Accuracy Sigmoid Kernel: 0.3855521727096596


In [25]:
cm_lin = confusion_matrix(y_test, linear_pred)
cm_poly = confusion_matrix(y_test, poly_pred)
cm_rbf = confusion_matrix(y_test, rbf_pred)
cm_sig = confusion_matrix(y_test, sig_pred)
print('Linear:\n', cm_lin)
print('Polynomial:\n', cm_poly)
print('RBF Kernel:\n', cm_rbf)
print('Sigmoid:\n', cm_sig)

Linear:
 [[6551  864   55]
 [ 972 3376  361]
 [  35  594 1644]]
Polynomial:
 [[7173  291    6]
 [ 375 4191  143]
 [   5  310 1958]]
RBF Kernel:
 [[7377   93    0]
 [ 156 4511   42]
 [  50   81 2142]]
Sigmoid:
 [[5416    0 2054]
 [3925    1  783]
 [2118    0  155]]


## Selecting the best model: Radial Basis Kernel

In [41]:
X = df[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X = pd.concat([X, pd.get_dummies(X.category, prefix='Cat_')], axis=1).drop('category', axis = 1)
y = df['groups']

In [42]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovr').fit(X, y)

## Using our best fit model to predict healthy recipes for test dataset

In [43]:
test_data = pd.read_csv("C:/Users/limst/Documents/github/Recipe_Recommender_System/Data/Raw Data/test_dataset_all.csv")

In [44]:
test_data.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,date,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,436255,beverages,gingered watermelon juice,2,10,78.2,1.0,63.0,1.0,3.0,...,2010-09-09,Very refreshing on a warm day of Indian summer...,5,4.359270,0.000000,4.143135,0.000000,1.098612,0.000000,1.791759
1,426940,pasta,pastitso macaroni and meat,15,90,1090.8,108.0,15.0,82.0,108.0,...,2010-09-09,"Well, LIG, after I dirtied every pot and pan i...",5,6.994667,4.682131,2.708050,4.406719,4.682131,5.214936,2.944439
2,77206,vegetables,almost boston market creamed spinach,10,50,337.9,43.0,9.0,24.0,18.0,...,2010-09-09,"I don't know about Boston Market, Kitten, but ...",5,5.822750,3.761200,2.197225,3.178054,2.890372,4.430817,1.609438
3,336781,vegetables,beef stir fry asian style,12,35,283.2,26.0,8.0,14.0,46.0,...,2010-09-09,"Quick, easy and delicious. I marinated the be...",5,5.646153,3.258097,2.079442,2.639057,3.828641,3.295837,1.098612
4,178213,seafood,garlic prawns in the microwave,7,35,257.0,29.0,1.0,29.0,34.0,...,2010-09-09,Divine sauce-the sherry is wonderful in this! ...,5,5.549076,3.367296,0.000000,3.367296,3.526361,2.708050,0.000000


In [45]:
X_test = test_data[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X_test = pd.concat([X_test, pd.get_dummies(X_test.category, prefix='Cat_')], axis=1).drop('category', axis = 1)

In [46]:
test_data['groups'] = rbf.predict(X_test)

In [47]:
test_data.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,review,rating,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,groups
0,436255,beverages,gingered watermelon juice,2,10,78.2,1.0,63.0,1.0,3.0,...,Very refreshing on a warm day of Indian summer...,5,4.359270,0.000000,4.143135,0.000000,1.098612,0.000000,1.791759,2
1,426940,pasta,pastitso macaroni and meat,15,90,1090.8,108.0,15.0,82.0,108.0,...,"Well, LIG, after I dirtied every pot and pan i...",5,6.994667,4.682131,2.708050,4.406719,4.682131,5.214936,2.944439,3
2,77206,vegetables,almost boston market creamed spinach,10,50,337.9,43.0,9.0,24.0,18.0,...,"I don't know about Boston Market, Kitten, but ...",5,5.822750,3.761200,2.197225,3.178054,2.890372,4.430817,1.609438,1
3,336781,vegetables,beef stir fry asian style,12,35,283.2,26.0,8.0,14.0,46.0,...,"Quick, easy and delicious. I marinated the be...",5,5.646153,3.258097,2.079442,2.639057,3.828641,3.295837,1.098612,1
4,178213,seafood,garlic prawns in the microwave,7,35,257.0,29.0,1.0,29.0,34.0,...,Divine sauce-the sherry is wonderful in this! ...,5,5.549076,3.367296,0.000000,3.367296,3.526361,2.708050,0.000000,1


In [48]:
test_data.to_csv("test_healthy.csv")